In [57]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report

In [10]:
df = pd.read_csv('Home_prices.csv')
df.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

Usuwanie brakujących wartości

In [18]:
df = df.dropna(axis=1)
print(f'Rozmiar: {df.size / df.columns.size} x {df.columns.size}')
df.head()

Rozmiar: 1460.0 x 62


,Id,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,12,2008,WD,Normal,250000


Wybór macierzy predykcji 'Y' i macierzy cech 'X'

In [26]:
y_data = df["SalePrice"]
for name, series in df.items():
    print(f'Nazwa: {name}. Ilosc unikatowych wartosci: {series.nunique()}')

Nazwa: Id ilosc unikatowych wartosci: 1460
Nazwa: MSSubClass ilosc unikatowych wartosci: 15
Nazwa: MSZoning ilosc unikatowych wartosci: 5
Nazwa: LotArea ilosc unikatowych wartosci: 1073
Nazwa: Street ilosc unikatowych wartosci: 2
Nazwa: LotShape ilosc unikatowych wartosci: 4
Nazwa: LandContour ilosc unikatowych wartosci: 4
Nazwa: Utilities ilosc unikatowych wartosci: 2
Nazwa: LotConfig ilosc unikatowych wartosci: 5
Nazwa: LandSlope ilosc unikatowych wartosci: 3
Nazwa: Neighborhood ilosc unikatowych wartosci: 25
Nazwa: Condition1 ilosc unikatowych wartosci: 9
Nazwa: Condition2 ilosc unikatowych wartosci: 8
Nazwa: BldgType ilosc unikatowych wartosci: 5
Nazwa: HouseStyle ilosc unikatowych wartosci: 8
Nazwa: OverallQual ilosc unikatowych wartosci: 10
Nazwa: OverallCond ilosc unikatowych wartosci: 9
Nazwa: YearBuilt ilosc unikatowych wartosci: 112
Nazwa: YearRemodAdd ilosc unikatowych wartosci: 61
Nazwa: RoofStyle ilosc unikatowych wartosci: 6
Nazwa: RoofMatl ilosc unikatowych wartosci: 8
N

In [45]:
num_features = ['MSSubClass', 'OverallQual',
                'YearBuilt', 'YearRemodAdd']
string_features = ['MSZoning', 'Neighborhood', 'Condition1', 'Condition2', 'ExterQual', 'Heating', 'CentralAir', 'KitchenQual']


In [40]:
for feature in string_features:
    print(f'{feature}: {df[feature].unique()}')

MSZoning: ['RL' 'RM' 'C (all)' 'FV' 'RH']
Neighborhood: ['CollgCr' 'Veenker' 'Crawfor' 'NoRidge' 'Mitchel' 'Somerst' 'NWAmes'
 'OldTown' 'BrkSide' 'Sawyer' 'NridgHt' 'NAmes' 'SawyerW' 'IDOTRR'
 'MeadowV' 'Edwards' 'Timber' 'Gilbert' 'StoneBr' 'ClearCr' 'NPkVill'
 'Blmngtn' 'BrDale' 'SWISU' 'Blueste']
Condition1: ['Norm' 'Feedr' 'PosN' 'Artery' 'RRAe' 'RRNn' 'RRAn' 'PosA' 'RRNe']
Condition2: ['Norm' 'Artery' 'RRNn' 'Feedr' 'PosN' 'PosA' 'RRAn' 'RRAe']
ExterQual: ['Gd' 'TA' 'Ex' 'Fa']
Heating: ['GasA' 'GasW' 'Grav' 'Wall' 'OthW' 'Floor']
CentralAir: ['Y' 'N']
KitchenQual: ['Gd' 'TA' 'Ex' 'Fa']


In [53]:
df_str_dummy = pd.get_dummies(data=df[string_features], drop_first=False)
X = df_str_dummy
X.head()
# df[num_features].size / df[num_features].columns.size, df_str_dummy.size / df_str_dummy.columns.size

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,...,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,CentralAir_N,CentralAir_Y,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA
0,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False
1,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,True
2,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False
3,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False
4,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False


In [54]:
X.describe()

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,...,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,CentralAir_N,CentralAir_Y,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA
count,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,...,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460
unique,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
top,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,True
freq,1450,1395,1444,1151,1242,1443,1458,1444,1402,1432,...,1442,1453,1458,1456,1365,1365,1360,1421,874,735


In [56]:
x_train, y_train, x_test, y_test = train_test_split(X, y_data, test_size=0.2, random_state=31)

In [58]:
model = RandomForestClassifier()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
print(accuracy, report)

ValueError: Found input variables with inconsistent numbers of samples: [1168, 292]